In [62]:
import numpy as np 
import pandas as pd
import os
import joblib
import sklearn 
import matplotlib
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

#Regressions:

from sklearn.multioutput import MultiOutputRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

#Metric
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn import tree

from pandas import DataFrame


# Show progress bar
from tqdm import tqdm

In [63]:
from dtreeviz.trees import dtreeviz # remember to load the package

In [64]:
df = pd.read_csv('dataset_augmented.csv')
df

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,...,x8_superhydride,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm,abs_nm,emission_nm,Unnamed: 78
0,0,0,0,0,2.798293,0.769563,-0.528015,1.951881,-0.431077,-0.641827,...,0,0,0,1,0,0,2.753333,599.0,612.000000,13.000000
1,1,1,1,1,0.055436,-0.361432,-0.528015,1.958217,-0.431077,-0.641827,...,0,0,0,1,0,0,2.600000,530.0,613.000000,83.000000
2,2,2,2,2,0.782458,0.017999,-0.528015,0.600491,-0.431077,-0.641827,...,0,0,0,1,0,0,3.500000,595.0,610.000000,15.000000
3,3,4,4,4,0.055436,0.280681,-0.528015,1.958217,-0.431077,-0.641827,...,0,0,0,1,0,0,2.520000,520.0,612.000000,92.000000
4,4,5,5,5,0.055436,0.280681,-0.528015,1.958217,-0.431077,-0.641827,...,0,0,0,1,0,0,2.660000,590.0,612.000000,22.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,212,214,214,214,-0.580709,-0.667895,-0.070987,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,5.433333,570.0,605.000000,35.000000
213,213,215,215,215,-0.580709,-0.667895,-0.070987,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,5.433333,562.0,605.000000,43.000000
214,214,216,216,216,-0.580709,-0.667895,-0.070987,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,5.433333,549.0,589.666667,40.666667
215,215,217,217,217,-0.423738,-0.631411,-0.201566,-0.304661,-0.085945,-0.600008,...,0,0,0,0,0,0,2.000000,523.0,597.000000,74.000000


In [65]:
# Input for ML models

input_col = ['in_amount_mmol',
             'p_amount_mmol',
             'sol_amount_ml',
             'TOP_amount_mmol',
             'acid_amount_mmol',
             'amine_amount_mmol',
             'thiol_amount_mmol',
             'zinc_amount_mmol',
             'other_amount_mmol',
             'total_volume_ml',
             'temp_c',
             'time_min',
             'x0_chloroindium oxalate',
             'x0_indium acetate',
             'x0_indium bromide',
             'x0_indium chloride',
             'x0_indium iodide',
             'x0_indium myristate',
             'x0_indium oxalate',
             'x0_indium palmitate',
             'x0_indium trifluoroacetate',
             'x0_indium tris(N,N-diisopropylacetamidinato)',
             'x1_bis(trimethylsilyl)phosphine',
             'x1_phosphine gas',
             'x1_phosphorus trichloride',
             'x1_sodium phosphide',
             'x1_tris(diethylamino)phosphine',
             'x1_tris(dimethylamino)phosphine',
             'x1_tris(trimethylgermyl)phosphine',
             'x1_tris(trimethylsilyl)phosphine',
             'x1_white phosphorus',
             'x2_4-ethylpyridine',
             'x2_None',
             'x2_dimethylformamide',
             'x2_mesitylene',
             'x2_octadecene',
             'x2_toluene',
             'x3_None',
             'x3_trioctylphosphine',
             'x4_None',
             'x4_lauric acid',
             'x4_myristic acid',
             'x4_oleic acid',
             'x4_palmitic acid',
             'x4_stearic acid',
             'x5_None',
             'x5_dioctylamine',
             'x5_dodecylamine',
             'x5_hexadecylamine',
             'x5_octylamine',
             'x5_oleylamine',
             'x6_None',
             'x6_dodecanethiol',
             'x7_None',
             'x7_zinc acetate',
             'x7_zinc bromide',
             'x7_zinc chloride',
             'x7_zinc iodide',
             'x7_zinc octanoate',
             'x7_zinc oleate',
             'x7_zinc stearate',
             'x7_zinc undecylenate',
             'x8_None',
             'x8_acetic acid',
             'x8_copper bromide',
             'x8_superhydride',
             'x8_oleic acid',
             'x8_tetrabutylammonium myristate',
             'x8_trioctylamine',
             'x8_trioctylphosphine oxide',
             'x8_water',
             'x8_zinc iodide'
            ]

#Three individual outputs:
diameter = ['diameter_nm']
emission = ['emission_nm']
absorbance = ['abs_nm']

#Splitting dataset

X = df[input_col]

Y_d = df[diameter]
Y_e = df[emission]
Y_a = df[absorbance]


X_train_d, X_test_d, Y_train_d, Y_test_d = train_test_split(X, Y_d, test_size=0.15, random_state=45, shuffle=True)
X_train_e, X_test_e, Y_train_e, Y_test_e = train_test_split(X, Y_e, test_size=0.15, random_state=45, shuffle=True)
X_train_a, X_test_a, Y_train_a, Y_test_a = train_test_split(X, Y_a, test_size=0.15, random_state=45, shuffle=True)

In [66]:
model_SO_diameter = joblib.load('model_SO_diameter_DecisionTree.joblib')
model_SO_emission = joblib.load('model_SO_emission_ExtraTrees.joblib')
model_SO_abs = joblib.load('model_SO_abs_DecisionTree.joblib')

model_MO = joblib.load('model_MO_DecisionTree.joblib')



/home/anhhaokudo/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/anhhaokudo/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator ExtraTreeRegressor from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/anhhaokudo/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator ExtraTreesRegressor from version 0.24.1 when using version 1.0.2. This might

In [13]:
text_representation = tree.export_text(model_SO_abs)
print(text_representation)

|--- feature_53 <= 0.50
|   |--- feature_5 <= -0.63
|   |   |--- feature_40 <= 0.50
|   |   |   |--- feature_15 <= 0.50
|   |   |   |   |--- feature_42 <= 0.50
|   |   |   |   |   |--- feature_9 <= -0.15
|   |   |   |   |   |   |--- feature_2 <= -0.23
|   |   |   |   |   |   |   |--- feature_58 <= 0.50
|   |   |   |   |   |   |   |   |--- value: [480.00]
|   |   |   |   |   |   |   |--- feature_58 >  0.50
|   |   |   |   |   |   |   |   |--- value: [466.33]
|   |   |   |   |   |   |--- feature_2 >  -0.23
|   |   |   |   |   |   |   |--- value: [436.00]
|   |   |   |   |   |--- feature_9 >  -0.15
|   |   |   |   |   |   |--- value: [500.00]
|   |   |   |   |--- feature_42 >  0.50
|   |   |   |   |   |--- feature_9 <= -0.14
|   |   |   |   |   |   |--- feature_4 <= -0.32
|   |   |   |   |   |   |   |--- value: [460.00]
|   |   |   |   |   |   |--- feature_4 >  -0.32
|   |   |   |   |   |   |   |--- feature_4 <= -0.32
|   |   |   |   |   |   |   |   |--- value: [457.00]
|   |   |   |   | 

In [30]:
s = Y_d.squeeze()
t = X.squeeze()
print(type(s))

<class 'pandas.core.series.Series'>


In [52]:
from dtreeviz.trees import dtreeviz # remember to load the package

viz = dtreeviz(model_SO_diameter, X, s,
                target_name="target",
                feature_names=input_col)

viz.save('dtreeviz.svg')

In [67]:
import graphviz 


In [47]:
dot_data = tree.export_graphviz(model_SO_diameter, out_file=None,
                               feature_names=input_col,
                                filled=True,
                                rounded=True,  
                              special_characters=True) 
graph = graphviz.Source(dot_data)
graph.render("2")
graph.save()

'Gini'

In [68]:
dot_data = tree.export_graphviz(model_MO, out_file=None,
                               feature_names=input_col,
                                filled=True,
                                rounded=True,  
                              special_characters=True) 
graph = graphviz.Source(dot_data)
graph.render("3")
graph.save("DT_2")

'DT_2'